# Lake model continued - sensitivity analysis

In the previous week you used the lake problem as a means of getting acquainted with the workbench. In this assignment we will continue with the lake problem, focussing explicitly on using it for open exploration. You can use the second part of the [open exploration tutorial](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html) for help.

**It is paramount that you are using the lake problem with 100 decision variables, rather than the one found on the website with the separate anthropogenic release decision**

There is substantial support in the ema_workbench for global sensitivity. For this, the workbench relies on [SALib](https://salib.readthedocs.io/en/latest/) and feature scoring which is a machine learning alternative for global sensitivity analysis.

## 1. SOBOL
1. Apply Sobol with 3 separate release policies (0, 0.05, and 0.1) and analyse the results for each release policy separately focusing on the reliability objective. Do the sensitivities change depending on the release policy? Can you explain why or why not?

*hint: you can use sobol sampling for the uncertainties, and set policies to a list with the 3 different release policies. Next, for the analysis using logical indexing on the experiment.policy column you can select the results for each separate release policy and apply sobol to each of the three separate release policies. If this sounds too complicated, just do it on each release policy separately.*

In [3]:
from ema_workbench import Samplers
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging, ScalarOutcome, Policy, MultiprocessingEvaluator)

from ema_workbench import Samplers

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

In [4]:
import math
import numpy as np
from scipy.optimize import brentq


def lake_problem(
        b=0.42,         # Decay parameter for P in lake (0.42 = irreversible)
        q=2.0,          # Recycling exponent
        mean=0.02,      # Mean of natural inflows
        stdev=0.0017,   # Future utility discount rate
        delta=0.98,     # Standard deviation of natural inflows
        alpha=0.4,      # Utility from pollution
        nsamples=100,   # Number of Monte Carlo samples to draw
        steps=100,      # Number of time steps
        l0=0, l1=0, l2=0, l3=0, l4=0, l5=0, l6=0, l7=0, l8=0, l9=0,
        l10=0, l11=0, l12=0, l13=0, l14=0, l15=0, l16=0, l17=0, l18=0, l19=0,
        l20=0, l21=0, l22=0, l23=0, l24=0, l25=0, l26=0, l27=0, l28=0, l29=0,
        l30=0, l31=0, l32=0, l33=0, l34=0, l35=0, l36=0, l37=0, l38=0, l39=0,
        l40=0, l41=0, l42=0, l43=0, l44=0, l45=0, l46=0, l47=0, l48=0, l49=0,
        l50=0, l51=0, l52=0, l53=0, l54=0, l55=0, l56=0, l57=0, l58=0, l59=0,
        l60=0, l61=0, l62=0, l63=0, l64=0, l65=0, l66=0, l67=0, l68=0, l69=0,
        l70=0, l71=0, l72=0, l73=0, l74=0, l75=0, l76=0, l77=0, l78=0, l79=0,
        l80=0, l81=0, l82=0, l83=0, l84=0, l85=0, l86=0, l87=0, l88=0, l89=0,
        l90=0, l91=0, l92=0, l93=0, l94=0, l95=0, l96=0, l97=0, l98=0, l99=0):
    # Create a NumPy array of the decision variables
    decisions = np.array([l0, l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11, l12, l13,
                          l14, l15, l16, l17, l18, l19, l20, l21, l22, l23, l24, l25,
                          l26, l27, l28, l29, l30, l31, l32, l33, l34, l35, l36, l37,
                          l38, l39, l40, l41, l42, l43, l44, l45, l46, l47, l48, l49,
                          l50, l51, l52, l53, l54, l55, l56, l57, l58, l59, l60, l61,
                          l62, l63, l64, l65, l66, l67, l68, l69, l70, l71, l72, l73,
                          l74, l75, l76, l77, l78, l79, l80, l81, l82, l83, l84, l85,
                          l86, l87, l88, l89, l90, l91, l92, l93, l94, l95, l96, l97,
                          l98, l99])
    nvars = len(decisions)

    # Calculate the critical pollution level (Pcrit)
    Pcrit = brentq(lambda x: x ** q / (1 + x ** q) - b * x, 0.01, 1.5)

    # Generate natural inflows using lognormal distribution
    natural_inflows = np.random.lognormal(
        mean=math.log(mean ** 2 / math.sqrt(stdev ** 2 + mean ** 2)),
        sigma=math.sqrt(math.log(1.0 + stdev ** 2 / mean ** 2)),
        size=(nsamples, nvars)
    )

    # Initialize the pollution level matrix X
    X = np.zeros((nsamples, nvars))

    # Loop through time to compute the pollution levels
    for t in range(1, nvars):
        X[:, t] = (1 - b) * X[:, t - 1] + (X[:, t - 1] ** q / (1 + X[:, t - 1] ** q)) + decisions[
            t - 1] + natural_inflows[:, t - 1]

    # Calculate the average daily pollution for each time step
    average_daily_P = np.mean(X, axis=0)

    # Calculate the reliability (probability of the pollution level being below Pcrit)
    reliability = np.sum(X < Pcrit) / float(nsamples * nvars)

    # Calculate the maximum pollution level (max_P)
    max_P = np.max(average_daily_P)

    # Calculate the utility by discounting the decisions using the discount factor (delta)
    utility = np.sum(alpha * decisions * np.power(delta, np.arange(nvars)))

    # Calculate the inertia (the fraction of time steps with changes larger than 0.02)
    inertia = np.sum(np.abs(np.diff(decisions)) > 0.02) / float(nvars - 1)

    return max_P, utility, inertia, reliability

In [5]:
model = Model('Lakeproblem', function=lake_problem)

model.uncertainties = model.uncertainties = [
    RealParameter("b", 0.1, 0.45),
    RealParameter("q", 2.0, 4.5),
    RealParameter("mean", 0.01, 0.05),
    RealParameter("stdev", 0.001, 0.005),
    RealParameter("delta", 0.93, 0.99),]

levers = []
for i in range(100):
    levers.append(RealParameter(f"l{i}", 0, 0.1))
    
model.levers = levers 

model.outcomes = [
    ScalarOutcome("max_P"),
    ScalarOutcome("utility"),
    ScalarOutcome("inertia"),
    ScalarOutcome("reliability"),
]
policy = Policy("no release", **{l.name:0 for l in model.levers})

In [ ]:
scenarios = 250*(2*5+2)
with MultiprocessingEvaluator(model, n_processes=-1) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios, policy, uncertainty_sampling = Samplers.SOBOL)

In [15]:
outcomes

{'max_P': array([0.08122279, 0.06972903, 0.08141988, ..., 9.22394095, 9.34978365,
        9.34886452]),
 'utility': array([0., 0., 0., ..., 0., 0., 0.]),
 'inertia': array([0., 0., 0., ..., 0., 0., 0.]),
 'reliability': array([1.    , 1.    , 1.    , ..., 0.5508, 0.3562, 0.3548])}

In [19]:
y = outcomes["max_P"]
problem = get_SALib_problem(model.uncertainties)
Si = sobol.analyze(problem, y, calc_second_order=True, print_to_console=True)

             ST   ST_conf
b      0.841812  0.042243
delta  0.000039  0.000020
mean   0.306484  0.033067
q      0.313808  0.031281
stdev  0.000203  0.000110
             S1   S1_conf
b      0.518634  0.056375
delta  0.000042  0.000206
mean   0.062771  0.026394
q      0.088464  0.031965
stdev  0.000342  0.001155
                      S2   S2_conf
(b, delta)      0.032809  0.078454
(b, mean)       0.112312  0.086726
(b, q)          0.080362  0.085449
(b, stdev)      0.032964  0.078523
(delta, mean)  -0.000097  0.000400
(delta, q)     -0.000059  0.000394
(delta, stdev) -0.000111  0.000374
(mean, q)       0.028607  0.036998
(mean, stdev)   0.017695  0.037420
(q, stdev)     -0.004587  0.041042


In [5]:
scenarios = 250*(2*5+2)
policy = Policy("no release", **{l.name:0.05 for l in model.levers})

experiments2, outcomes2 = perform_experiments(model, scenarios, policy, uncertainty_sampling = Samplers.SOBOL)

C:\Users\rvrij\anaconda\envs\Model_based_decisions\Lib\site-packages\scipy\stats\_qmc.py:763: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
100%|████████████████████████████████████| 36000/36000 [06:16<00:00, 95.53it/s]


In [ ]:
y = outcomes["max_P"]
problem = get_SALib_problem(model.uncertainties)
Si = sobol.analyze(problem, y, calc_second_order=True, print_to_console=True)

In [ ]:
scenarios = 250*(2*5+2)
policy = Policy("no release", **{l.name:0.1 for l in model.levers})

experiments3, outcomes3 = perform_experiments(model, scenarios, policy, uncertainty_sampling = Samplers.SOBOL)

In [ ]:
y = outcomes["max_P"]
problem = get_SALib_problem(model.uncertainties)
Si = sobol.analyze(problem, y, calc_second_order=True, print_to_console=True)

## 2. Feature scoring
Repeat the above analysis for the 3 release policies but now with extra trees [feature scoring](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html) and for all outcomes of interest. As a bonus, use the sobol experiment results as input for extra trees, and compare the results with those resulting from latin hypercube sampling.

*hint: you can use [seaborn heatmaps](https://seaborn.pydata.org/generated/seaborn.heatmap.html) for a nice figure of the results. See also the [features scoring](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html#feature-scoring) section of the tutorial.*


In [ ]:
from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.feature_scoring import f_regression

In [ ]:
import pandas as pd
x = pd.DataFrame(experiments) #turn into dataframe
y = outcomes['max_P'] 
y.shape

In [ ]:
fs = feature_scoring.get_ex_feature_scores(x, y, mode=RuleInductionType.REGRESSION, nr_trees=100, max_features=0.6)